Works with `d05199b`

### This notebook will walk through plotting EEG and related data using the tbd_eeg repository.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy import signal

# First import useful packages: os is a package for handling directories;
# numpy for math; matplotlib for plotting; and scipy for signal processing.

In [2]:
from allensdk.brain_observatory.sync_dataset import Dataset

# AllenSDK is a repo with tons of code for analyzing most of the
# AI-generated data. We'll use the Dataset class for extracting metadata
# from the sync h5 file.

In [3]:
from tbd_eeg.data_analysis.eegutils import EEGexp

# This imports the EEGexp class from the eegutils.py code.

##### This is called a "magic command." It is special to jupyter notebooks, and allows you to interact with plots.

In [4]:
%matplotlib widget

####  Enter folder and get metadata for experiment.

In [5]:
data_folder = r"/allen/programs/braintv/workgroups/nc-ophys/Leslie/eeg_pilot/mouse507190/pilot1_2020-02-28_10-33-11/recording1"
exp = EEGexp(data_folder)

In [6]:
# Let's print some meta data
print('Mouse number: ' + exp.mouse)
print('EEG sample rate: %d Hz' % exp.sample_rate)
print('To convert bits to microvolts, multiply by %0.3f.' % exp.bits_to_uvolts)

Mouse number: 507190
EEG sample rate: 10000 Hz
To convert bits to microvolts, multiply by 0.195.


#### Use EEGexp.memmap_EEGdata() function to load EEG data as a memmap matrix.

In [7]:
EEGdata = exp.memmap_EEGdata()
## this memmaps the data in the correct NN ch order, but does not convert it from bits to uV

Loading /allen/programs/braintv/workgroups/nc-ophys/Leslie/eeg_pilot/mouse507190/pilot1_2020-02-28_10-33-11/recording1/continuous/Rhythm_FPGA-111.0/continuous.dat


#### Plot data from one electrode for sanity check.

In [8]:
# Make an array for x-axis in seconds
eegtime = np.arange(0, len(EEGdata))/exp.sample_rate

In [9]:
plot_ch = 27 # choose which electrode to plot (zero-indexed, ch 30:31 do not exist)

# load and convert single ch data
single_ch = EEGdata[:,plot_ch] * exp.bits_to_uvolts

In [10]:
fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(eegtime, single_ch/1000.)

# make plot more readable
ax.set_xlim((0, eegtime[-1]))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Raw signal (mV)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Raw signal (mV)')

#### Load the EEG timestamps_master_clock for combined plot at the end.

In [11]:
eegtime_master = np.load(exp.eegtimestamps_file)

#### Load .sync file as a Dataset (defined in AllenSDK)

In [12]:
sync_data = exp.load_sync_dataset()

In [13]:
linear_velocity, runtime = exp.load_running(sync_data)

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/allensdk/brain_observatory/sync_dataset.py:454: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return eval(self.dfile['analog_meta'].value)


In [14]:
# plot linear_velocity for whole session
fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(runtime, linear_velocity)
ax.plot(runtime, np.zeros_like(linear_velocity), "--", color="gray") # plots a gray line at zero

ax.set_title('Mouse %s linear velocity' % exp.mouse)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Velocity (cm/s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Velocity (cm/s)')

##### It's still a messy signal...we could try smoothing it?

In [15]:
# simple mean smoothing filter
smooth_velocity = np.zeros_like(linear_velocity)

# implement the running mean filter
k = 20 # filter window is actually k*2+1
for i in range(k, len(linear_velocity)-k-1):
    # each point is the average of k surrounding points
    smooth_velocity[i] = np.mean(linear_velocity[i-k:i+k])

In [16]:
fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(runtime, linear_velocity, label='original')
ax.plot(runtime, smooth_velocity, color='orange', label='smoothed') # plots smoothed linear velocity

ax.set_title('Mouse %s linear velocity' % exp.mouse)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Velocity (cm/s)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### Seems better...

#### Let's look at the isoflurane level.

In [17]:
iso_level, isotime = exp.load_analog_iso(sync_data) # still need to pass it the Dataset instance

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/allensdk/brain_observatory/sync_dataset.py:454: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return eval(self.dfile['analog_meta'].value)


In [18]:
fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(isotime, iso_level)

ax.set_title('Mouse %s isoflurane level' % exp.mouse)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Isoflurane level (%)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Isoflurane level (%)')

#### Can we extract the on/off times?

In [19]:
iso5inds = np.where(iso_level > 4.90)[0]
iso5_times = isotime[np.array([iso5inds[0], iso5inds[-1]])]
print(iso5_times)

[ 645.712 1189.232]


In [20]:
iso2inds = np.where((iso_level > 1.80) & (iso_level < 2.2))[0]
iso2inds = iso2inds[iso2inds > iso5inds[-1]] # removes iso2 level during initial increase to iso5
iso2_times = isotime[np.array([iso2inds[0], iso2inds[-1]])]
print(iso2_times)

[1194.048 1867.552]


In [21]:
fig, ax = plt.subplots(figsize=(10, 2))
ax.plot(isotime, iso_level)
ax.axvspan(iso5_times[0], iso5_times[1], facecolor='r', alpha=0.5)
ax.axvspan(iso2_times[0], iso2_times[1], facecolor='r', alpha=0.2)

ax.set_title('Mouse %s isoflurane level' % exp.mouse)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Isoflurane level (%)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Isoflurane level (%)')

### Plot all three together in same clock time

In [22]:
fig, axs = plt.subplots(3, 1, figsize=(10, 5), sharex=True)

axs[0].plot(isotime, iso_level, color='red')
axs[0].set_xlim((0, isotime[-1]))
axs[0].set_ylabel('Iso Level (%)')

axs[1].plot(runtime, smooth_velocity, color='orange')
axs[1].set_ylabel('Linear Velocity (cm/s)')

axs[2].plot(eegtime_master, single_ch/1000.)
axs[2].set_ylim((-2, 2))
axs[2].set_xlabel('Time (s)')
axs[2].set_ylabel('EEG (mV)')

fig.suptitle('Mouse %s' % exp.mouse)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Mouse 507190')

### Spectrogram plot

In [23]:
f, t, Sxx = signal.spectrogram(single_ch, exp.sample_rate, nperseg=2**13, scaling='density')

In [24]:
frinds = f < 100
print(np.mean(np.mean(Sxx[frinds,:], axis=1)))

25.818365940779437


In [25]:
fig, axs = plt.subplots(2, 1, figsize=(8,5), sharex=True)

# plot raw signal
axs[0].plot(eegtime, single_ch/1000.)
axs[0].set_xlim((0, eegtime[-1]))
axs[0].set_ylim((-2, 2))
# axs[0].axvspan(ind_beg, ind_end, facecolor='r', alpha=0.5)
# axs[0].axvspan(ind_end, main_end, facecolor='r', alpha=0.2)
# axs[0].set_xlabel('Time (s)')
axs[0].set_ylabel('EEG (mV)')

# plot spectrogram
spgm = axs[1].pcolormesh(t, f[frinds], Sxx[frinds,:], cmap='YlOrRd', vmin=0, vmax=40)
axs[1].set_ylabel('Frequency (Hz)')
axs[1].set_xlabel('Time (s)')

# do the colorbar :(
fig.subplots_adjust(bottom=0.1, right=0.85, top=0.9, hspace=0.2)
cax = plt.axes([0.9, 0.1, 0.025, 0.36])
cbar = fig.colorbar(spgm, cax=cax)
cbar.ax.set_ylabel('PSD (V**2/Hz)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'PSD (V**2/Hz)')

#### Should we normalize the spectrum?

In [26]:
print(len(f))
print(np.shape(Sxx))
print(np.argmax(Sxx[1,:]))

4097
(4097, 5039)
1654


In [27]:
# Sxx[:,825:830] = np.zeros_like(Sxx[:,825:830])
Sxx[:,1652:1657] = np.zeros_like(Sxx[:,1652:1657])

# did we just remove the 60Hz?

In [28]:
normed_array = np.zeros_like(Sxx)
for row_idx in range(np.shape(Sxx)[0]):
    row = Sxx[row_idx,:]
    
    minval = row.min()
    maxval = row.max()

    normed_array[row_idx,:] = (row - minval)/(maxval - minval)

In [29]:
spectime = t + eegtime_master[0]

In [30]:
fig = plt.figure(figsize=(10,5)) #, constrained_layout=True)

gs = GridSpec(5, 1, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1:3, :], sharex=ax1)
ax3 = fig.add_subplot(gs[3:5, :], sharex=ax1)

# plot running
ax1.plot(runtime, smooth_velocity, color='green')
ax1.set_xlim((0, runtime[-1]))
ax1.set_ylabel('Velocity (cm/s)')
ax1.axvspan(iso5_times[0], iso5_times[1], facecolor='r', alpha=0.3)
ax1.axvspan(iso2_times[0], iso2_times[1], facecolor='r', alpha=0.1)
plt.setp(ax1.get_xticklabels(), visible=False)

# plot raw signal
ax2.plot(eegtime_master, single_ch/1000.)
ax2.set_ylim((-2, 2))
ax2.axvspan(iso5_times[0], iso5_times[1], facecolor='r', alpha=0.3)
ax2.axvspan(iso2_times[0], iso2_times[1], facecolor='r', alpha=0.1)
ax2.set_ylabel('EEG (mV)')
plt.setp(ax2.get_xticklabels(), visible=False)

# plot spectrogram
spgm = ax3.pcolormesh(spectime, f[frinds], normed_array[frinds,:], cmap='YlOrRd', vmin=0, vmax=1)
ax3.set_ylabel('Freq (Hz)')
ax3.set_xlabel('Time (s)')

# do the colorbar :(
fig.subplots_adjust(bottom=0.1, right=0.9, top=0.9, hspace=0.2)
cax = plt.axes([0.92, 0.1, 0.02, 0.31])
cbar = fig.colorbar(spgm, cax=cax)
cbar.ax.set_ylabel('norm PSD')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'norm PSD')

#### Plot smaller window?

In [37]:
window_beg = 80. # time of interest in seconds
pl_win_len = 20. # window length in seconds

In [38]:
runinds = np.argwhere((runtime > window_beg) & (runtime < window_beg + pl_win_len)).flatten()
eeginds = np.argwhere((eegtime_master > window_beg) & (eegtime_master < window_beg + pl_win_len)).flatten()
specinds = np.argwhere((spectime > window_beg) & (spectime < window_beg + pl_win_len)).flatten()

In [39]:
newnormspec = normed_array[:,specinds]

In [40]:
fig = plt.figure(figsize=(6,5)) #, constrained_layout=True)

gs = GridSpec(5, 1, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1:3, :], sharex=ax1)
ax3 = fig.add_subplot(gs[3:5, :], sharex=ax1)

# plot running
ax1.plot(runtime[runinds], smooth_velocity[runinds], color='green')
ax1.set_xlim((window_beg, window_beg+pl_win_len))
ax1.set_ylabel('Velocity (cm/s)')
ax1.set_ylim((-1, 30))
# ax1.axvspan(iso5_times[0], iso5_times[1], facecolor='r', alpha=0.3)
# ax1.axvspan(iso2_times[0], iso2_times[1], facecolor='r', alpha=0.1)
plt.setp(ax1.get_xticklabels(), visible=False)

# plot raw signal
ax2.plot(eegtime_master[eeginds], single_ch[eeginds]/1000., linewidth=1)
ax2.set_ylim((-0.5, 0.5))
# ax2.axvspan(iso5_times[0], iso5_times[1], facecolor='r', alpha=0.3)
# ax2.axvspan(iso2_times[0], iso2_times[1], facecolor='r', alpha=0.1)
ax2.set_ylabel('EEG (mV)')
plt.setp(ax2.get_xticklabels(), visible=False)

# plot spectrogram
spgm = ax3.pcolormesh(spectime[specinds], f[frinds], newnormspec[frinds,:], cmap='YlOrRd', vmin=0, vmax=1)
ax3.set_ylabel('Freq (Hz)')
ax3.set_xlabel('Time (s)')

# # do the colorbar :(
# fig.subplots_adjust(bottom=0.1, right=0.9, top=0.9, hspace=0.2)
# cax = plt.axes([0.92, 0.1, 0.02, 0.31])
# cbar = fig.colorbar(spgm, cax=cax)
# cbar.ax.set_ylabel('norm PSD')

fig.suptitle('Awake')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Awake')

### Plot multiple channels and spectrogram for small time windows.

In [41]:
# channels to remove from plot
badchs = np.array([30, 31])

In [42]:
# allplchs = np.arange(0,30)
allplchs = np.array([0, 6, 11, 20, 26])

In [43]:
window_starts = np.array([40., 1399., 2848.]) # times of interest in seconds
pl_win_len = 1. # window length in seconds

sep = 500
xtime = np.linspace(0, 2, int(pl_win_len*exp.sample_rate))

In [44]:
fig, axs = plt.subplots(1, len(window_starts), sharex=True, sharey=True, figsize=(10,6))

for i, winstart in enumerate(window_starts):
    aind = int(winstart*exp.sample_rate)
    bind = aind + int(pl_win_len*exp.sample_rate)
    
    for k, plch in enumerate(allplchs):
        
#         axs[i].plot(xtime, smooth(EEGdata[aind:bind,plch])*exp.bitVolts + sep*k)
        axs[i].plot(xtime, EEGdata[aind:bind,plch]*exp.bits_to_uvolts + sep*k)
        
    axs[i].set_xlim((0, pl_win_len))
    axs[i].set_ylim((-sep+50, sep*(k+1)+50))
    
axs[0].set_yticks(np.arange(0, sep*len(allplchs), sep))
axs[0].set_yticklabels(allplchs)
axs[0].set_ylabel('EEG electrode')
axs[0].set_title('Awake')
axs[0].set_xlabel('Time (s)')

axs[1].tick_params(axis='y', labelleft=False)
plt.setp(axs[1].spines.values(), color='r')
plt.setp([axs[1].get_xticklines(), axs[1].get_yticklines()], color='r')
axs[1].set_title('Anesthetized')
axs[1].set_xlabel('Time (s)')

axs[2].tick_params(axis='y', labelleft=False)
axs[2].set_title('Recovery')
axs[2].set_xlabel('Time (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')